# Web Scraping with Selenium

Importación de las librerías

In [130]:
#Para la manipulación de datos
import pandas as pd

#Servicio y driver de Chrome de Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#Las opciones que vamos a tener para buscar elementos
from selenium.webdriver.common.by import By

#Para esperar a que carguen las páginas
import time


Creamos el driver para controlar el navegador

In [131]:
service=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\Bustos_Traceur\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [137]:
url = "https://www.filmaffinity.com"
driver.get(url)

Aceptamos el pop-up de ser necesario

In [133]:
boton = driver.find_element(by=By.CLASS_NAME, value="css-v43ltw")
boton.click()

Hacemos una función que devuelva en un diccionario todos los datos de las películas, salvo la fecha de estreno y la url

Parámetros: url y fecha de estreno
Salida: Diccionario con todos los datos

In [235]:
def get_datos_pelicula(driver, url, estreno):

    #Creamos el diccionario
    datos_peli = dict()

    #Añadimos al diccionario los parámetros que nos vienen dados
    datos_peli['url'] = url
    datos_peli['fecha_estreno'] = estreno

    #Accedemos a la url
    driver.get(url)

    #Sacamos los datos principales
    principales = driver.find_element(By.CSS_SELECTOR, value='div#left-column dl.movie-info')

    claves = principales.find_elements(by=By.CSS_SELECTOR, value='dt')
    valores = principales.find_elements(by=By.CSS_SELECTOR, value='dd')

    for i in range(len(claves)):
        datos_peli[claves[i].text] = valores[i].text


    #Sacamos la nota media
    
    
    notamedia = driver.find_elements(by= By.ID, value='movie-rat-avg')
    if len(notamedia) > 0:
        notamedia1 = driver.find_elements(by= By.ID, value='movie-rat-avg')
        datos_peli['nota_media']= notamedia1[0].text

    #Sacamos la cantidad de votos
    votos = driver.find_elements(By.CSS_SELECTOR, value='#movie-count-rat span')
    if len(votos) > 0:
        votos1 = driver.find_element(By.CSS_SELECTOR, value='#movie-count-rat span').text
        datos_peli['votos']= votos1

    #Críticas profesionales
    n_criticas = driver.find_elements(by=By.CSS_SELECTOR, value="#right-column > a > div > div.body > div > div.legend-wrapper .leg")

    if len(n_criticas) > 0:
        #Criticas profesionales positivas
        datos_peli['positivas'] = n_criticas[0].text

        #Criticas profesionales regulares
    
        datos_peli['regulares'] = n_criticas[1].text

        #Criticas profesionales negativas
    
        datos_peli['negativas'] = n_criticas[2].text

    #Cinco primeras criticas
    criticas = driver.find_elements(by=By.CSS_SELECTOR, value="ul#pro-reviews li")
    
    i = 0
    while i < 5 and len(n_criticas):
        critica = criticas[i].find_element(by=By.CSS_SELECTOR, value="div a div").text
        datos_peli['critica_'+str(i+1)] = critica
        i += 1


    return datos_peli

In [185]:

test = get_datos_pelicula(driver, "https://www.filmaffinity.com/es/film599984.html", 12)
test

{'url': 'https://www.filmaffinity.com/es/film599984.html',
 'fecha_estreno': 12,
 'Título original': 'Der Untergang',
 'Año': '2004',
 'Duración': '155 min.',
 'País': ' Alemania',
 'Dirección': 'Oliver Hirschbiegel',
 'Guion': 'Bernd Eichinger. Libro: Joaquim Fest',
 'Música': 'Stephan Zacharias',
 'Fotografía': 'Rainer Klausmann',
 'Reparto': 'Bruno Ganz, Alexandra Maria Lara, Corinna Harfouch, Ulrich Matthes, Juliane Köhler, Heino Ferch, Christian Berkel, Matthias Habich, Thomas Kretschmann, Rolf Kanies, Christian Redl, Elizaveta Boyarskaya, Julia Jentsch',
 'Productora': 'Constantin Film',
 'Género': 'Drama. Bélico | Nazismo. II Guerra Mundial. Histórico',
 'Sinopsis': 'Berlín, abril de 1945. La guerra está sentenciada, pero en las calles de la capital del Reich todavía se libra una encarnizada batalla. Adolf Hitler (Bruno Ganz) y sus fieles se han atrincherado en un búnker. Entre ellos se encuentra Traudl Junge (Alexandra Maria Lara), la secretaria personal del Führer. En el exter

Probamos la función que hemos hecho. Aquí tienes un enlace de prueba: https://www.filmaffinity.com/es/film599984.html

Entramos en el link de las películas en cartelera

In [230]:
link = driver.find_element(by=By.LINK_TEXT, value="Películas en cartelera").click()


Sacamos todas las películas y llamamos a la función con cada película

In [231]:
rows = driver.find_elements(By.CSS_SELECTOR, value="#wrapper-cat .row")
links = []
for row in rows:
    peliculas = row.find_elements(by=By.CSS_SELECTOR, value=".movie-poster")

    for pelicula in peliculas:
        url = pelicula.find_element(by=By.TAG_NAME, value="a").get_attribute("href")
        estreno = pelicula.find_element(by=By.CSS_SELECTOR, value="a .release-text").text
        link_pelicula={
            "url" : url,
            "estreno" : estreno.replace("\n", " ")
        }
        links.append(link_pelicula)
links

[{'url': 'https://www.filmaffinity.com/es/film427155.html',
  'estreno': '13 may.'},
 {'url': 'https://www.filmaffinity.com/es/film836349.html',
  'estreno': '13 may.'},
 {'url': 'https://www.filmaffinity.com/es/film398907.html',
  'estreno': '13 may.'},
 {'url': 'https://www.filmaffinity.com/es/film691906.html',
  'estreno': '13 may.'},
 {'url': 'https://www.filmaffinity.com/es/film600773.html',
  'estreno': '13 may.'},
 {'url': 'https://www.filmaffinity.com/es/film355004.html',
  'estreno': '13 may.'},
 {'url': 'https://www.filmaffinity.com/es/film648099.html',
  'estreno': '13 may.'},
 {'url': 'https://www.filmaffinity.com/es/film962586.html',
  'estreno': '13 may.'},
 {'url': 'https://www.filmaffinity.com/es/film772620.html',
  'estreno': '13 may.'},
 {'url': 'https://www.filmaffinity.com/es/film686532.html',
  'estreno': '13 may.'},
 {'url': 'https://www.filmaffinity.com/es/film692875.html',
  'estreno': '13 may.'},
 {'url': 'https://www.filmaffinity.com/es/film510672.html',
  'es

Ahora usamos los links para llamar a la funcion y sacar los datos

In [236]:
#Reservamos una variable para el df
df = None

#Bucle para iterar sobre links
for link in links:
    datos_pelicula = get_datos_pelicula(driver, link['url'], link['estreno'])
    if df is None:
        df = pd.DataFrame(columns=datos_pelicula.keys())
    df = df.append(datos_pelicula, ignore_index=True)
    print(f"Añadida {datos_pelicula['Título original']}")
    

Añadida Firestarter
Añadida Father Stu
Añadida El universo de Óliver
Añadida La maniobra de la tortuga
Añadida En attendant Bojanglesaka
Añadida Presqueaka
Añadida Ennio: The Maestro
Añadida Un autre mondeaka
Añadida Die Wannseekonferenzaka
Añadida Eles transportan a morte
Añadida Mamma Mu hittar hemaka
Añadida Jaddeh Khaki
Añadida Val
Añadida Devolvendo o golpe
Añadida Jayeshbhai Jordaar
Añadida Saunkan Saunkne
Añadida Salvajes, el cuento del lobo
Añadida Doctor Strange in the Multiverse of Madness
Añadida Red Rocket
Añadida Il peccato
Añadida Onoda, 10.000 nuits dans la jungleaka
Añadida The Storms of Jeremy Thomas
Añadida Culpa
Añadida Alcarràs
Añadida Downton Abbey: A New Era
Añadida X
Añadida Mystèreaka
Añadida Where Is Anne Frank
Añadida Go Fish
Añadida Le Monde d'hier
Añadida La jefa
Añadida The Northman
Añadida Veneciafrenia
Añadida Dog
Añadida Tendre et saignantaka
Añadida Arthur Rambo
Añadida El beso de Dios. El documental de la Misa
Añadida Ambulance
Añadida The Lost Cityaka

In [226]:
df

,url,fecha_estreno,Título original,Año,Duración,País,Dirección,Guion,Música,Fotografía,...,votos,positivas,regulares,negativas,critica_1,critica_2,critica_3,critica_4,critica_5,
0,https://www.filmaffinity.com/es/film427155.html,13 may.,Firestarter,2022,94 min.,Estados Unidos,Keith Thomas,Scott Teems. Novela: Stephen King,"John Carpenter, Cody Carpenter, Daniel A. Davies",Karim Hussain,...,287,3,11,16,"""Podría ser una película de superhéroes ciegos...","""Insulsa, áspera e incluso cómica. (...) Ni la...","""A pesar de las sólidas interpretaciones de Za...","""Da la sensación de que se han extirpado trozo...","""Cumple con los requisitos, pero por los pelos...",NaN
1,https://www.filmaffinity.com/es/film836349.html,13 may.,Father Stu,2022,124 min.,Estados Unidos,Rosalind Ross,Rosalind Ross,Dickon Hinchliffe,Jacques Jouffret,...,49,4,11,7,"""Es grata de ver por la chispa de Wahlberg (.....","""Más cercana a un reality de la MTV mal traduc...","""Wahlberg se deja, y hablamos de forma literal...","""Es una película sorprendentemente sincera sob...","""Esta producción es anodina y superficial (......",NaN
2,https://www.filmaffinity.com/es/film398907.html,13 may.,El universo de Óliver,2022,112 min.,España,Alexis Morante,"Alexis Morante, Miguel Ángel González, Raúl Sa...",Julio de la Rosa,Carlos García de Dios,...,68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.filmaffinity.com/es/film691906.html,13 may.,La maniobra de la tortuga,2022,103 min.,España,Juan Miguel del Castillo,"Juan Miguel del Castillo, José Rodríguez. Libr...",Xavier Font,Gina Ferrer,...,110,2,5,1,"""Película irregular (...) le cuesta arrancar, ...","""Acaso pueda reprochársele al director una cie...","""Hay mucho de cine social y denuncia sobre la ...","""Del Castillo (...) en demasiadas ocasiones ti...","""Es un thriller feroz, intenso, que sabe aprov...",NaN
4,https://www.filmaffinity.com/es/film600773.html,13 may.,En attendant Bojanglesaka,2021,126 min.,Francia,Régis Roinsard,"Romain Compingt, Régis Roinsard. Novela: Olivi...","Clare Manchon, Olivier Manchon",Guillaume Schiffman,...,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
5,https://www.filmaffinity.com/es/film355004.html,13 may.,Presqueaka,2021,92 min.,Francia,Bernard Campan,"Bernard Campan, Alexandre Jollien, Helene Grem...",Niklas Paschburg,Christophe Offenstein,...,66,5,1,0,"""Tierna, simpática, agradable, bienintencionad...","""Película feliz (...) luminosa (...) Desde el ...","""Muestra sin ser aleccionadora ni didáctica có...","""Reivindica la diferencia y lo hace a través d...","""Deliciosa (...) Atravesada por el aire meridi...",
6,https://www.filmaffinity.com/es/film648099.html,13 may.,Ennio: The Maestro,2021,156 min.,Italia,Giuseppe Tornatore,Giuseppe Tornatore,Ennio Morricone,"Giancarlo Leggeri, Fabio Zamarion",...,164,9,2,1,"""Se trata de un homenaje, de una lección de vi...","""Bonito documental (...) Tornatore no ha olvid...","""Tornatore ha conseguido poesía, eternidad, po...","""Es una película repetitiva, pesada y muy conv...","""El documental va rico en anécdotas, incluso t...",NaN
7,https://www.filmaffinity.com/es/film962586.html,13 may.,Un autre mondeaka,2021,96 min.,Francia,Stéphane Brizé,"Stéphane Brizé, Olivier Gorce",Camille Rocailleux,Eric Dumont,...,86,11,1,0,"""La primera secuencia del relato es un prodigi...","""El director hace que cada plano directamente ...","""Lindon interpreta al director de una fábrica ...","""Lindon ejerce (...) de Vincent Lindon, es dec...","""Viendo padecer a Lindon, una apuesta llevada ...",
8,https://www.filmaffinity.com/es/film772620.html,13 may.,Die Wannseekonferenzaka,2022,108 min.,Alemania,Matti Geschonneck,"Magnus Vattrodt, Paul Mommertz",NaN,Theo Bierkens,...,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
9,https://www.filmaffinity.com/es/film686532.html,13 may.,Eles transportan a morte,2021,76 min.,España,"Helena Girón, Samuel M. Delgado","Samuel M. Delgado, Helena Girón",Camilo Sanabria,José Ángel Alayón,...,135,7,0,0,"""Una poderosa indagación experimental sobr